In [1]:
# ! pip install IPython==7.31.1
! pip install geemap



     |████████████████████████████████| 1.9 MB 11.9 MB/s 
     |████████████████████████████████| 95 kB 3.2 MB/s 
     |████████████████████████████████| 3.3 MB 67.8 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 5.4 MB 30.5 MB/s 
     |████████████████████████████████| 98 kB 7.7 MB/s 
     |████████████████████████████████| 216 kB 47.6 MB/s 
     |████████████████████████████████| 99 kB 8.4 MB/s 
     |████████████████████████████████| 130 kB 80.8 MB/s 
     |████████████████████████████████| 1.2 MB 70.2 MB/s 
     |████████████████████████████████| 1.7 MB 56.2 MB/s 
     |████████████████████████████████| 8.5 MB 49.3 MB/s 
     |████████████████████████████████| 44 kB 2.5 MB/s 
     |████████████████████████████████| 1.3 MB 68.7 MB/s 
     |████████████████████████████████| 176 kB 52.4 MB/s 
     |████████████████████████████████| 97 kB 5.6 MB/s 
     |████████████████████████████████| 428 kB 64.6 MB/s 
     |████████████████████

In [2]:
import ee
import geemap
import ipywidgets as widgets
from ipywidgets import Layout
from datetime import datetime
from ipyleaflet import WidgetControl

ee.Authenticate()
ee.Initialize()

# ******* This script is utilized to visualize classification result ******
# ****** Created by Quandi, 2022/02/24 ******

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=wh6Z5VugrqWUE-JfQUyADfprZaZsQPQLaVPdIz9KeO8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi4s6RnSoMfu34Yjl839pBV04q30ax505baDbkhT1rFqlGOoMeuS3I

Successfully saved authorization token.


In [7]:
# ##### the functions used below ######
# --------------------------------------------------------------------------------------------------------------------------------------
# *** Function to mask clouds using the Sentinel-2 QA band. ***
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
  return image.updateMask(mask).divide(10000) \
        .select("B.*") \
        .copyProperties(image, ["system:time_start", "PRODUCT_ID"]) 

def obtainS2Dataset(study_area, date_start, date_end):
  s2_dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
          .filterBounds(study_area) \
          .filterDate(date_start, date_end) \
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)) \
          .map(maskS2clouds) \
          .select('B.*')

  return s2_dataset 
# --------------------------------------------------------------------------------------------------------------------------------------         

In [8]:
# ****** widgets of interactive system

startDate = widgets.DatePicker(
    description = 'Start date: ', 
    min = datetime(2017, 3, 28),
    value = datetime(2020, 2, 15), 
    disabled=False)
endDate = widgets.DatePicker(
    description='End date: ',
    min = datetime(2017, 3, 28),
    value = datetime(2020, 3, 15),
    disabled=False)

submit = widgets.Button(description='Submit', button_style='primary', tooltip='Click me')
style = {'description_width': 'initial'}

# *** checkbox widgets ***
checkbox_mask = widgets.Checkbox(
    value = False, 
    description = "Apply maize mask",
    style = style)
checkbox_split = widgets.Checkbox(
    value = False, 
    description = "Split window",
    style = style)

first_image = widgets.Dropdown(
    description='First image:',
    options=[
        'Red/Green/Blue',
        'NIR/Red/Green',
        'NIR/SWIR1/SWIR2',
        'NDVI',
        'EVI',
        'LSWI',
        'None'
    ],
    style = style,
    value='Red/Green/Blue'
)
second_image = widgets.Dropdown(
    description='Second image:',
    options=[
        'Maize classification',
        'Yield result',
        'None'
    ],
    value = 'None',
    style = style,
    # disabled = True
)
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')


In [9]:
# --------------------------------------------------------------------------------------------------------------------------------------
# ****** this function is used to display the first image on a specific map ******
def obtain_FirImage():
  vi_index = first_image.value
  if vi_index != None:
    date_start = ee.Date(str(startDate.value).split()[0])
    date_end = ee.Date(str(endDate.value).split()[0])
    
    if vi_index == "NDVI":
      def vi_func(image):
        return image.normalizedDifference(['B8', 'B4']).rename(vi_index)
      vi_palatte = {'min':0, 'max':1, 'palette': ['ffffff', '0b6b07']}
      vi_bands = vi_index
    elif vi_index == "Red/Green/Blue":
      def vi_func(image):
        return image.select(['B4', 'B3', 'B2'])
      vi_palatte = {'min':0, 'max':0.3, 'bands':['B4', 'B3', 'B2']}
      vi_bands = ['B4', 'B3', 'B2']
    elif vi_index == "NIR/Red/Green":
      def vi_func(image):
        return image.select(['B8', 'B4', 'B3'])
      vi_palatte = {'min':0, 'max':0.3, 'bands':['B8', 'B4', 'B3']}
      vi_bands = ['B8', 'B4', 'B3']
    elif vi_index == "NIR/SWIR1/SWIR2":
      def vi_func(image):
        return image.select(['B8', 'B11', 'B12'])
      vi_palatte = {'min':0, 'max':0.3, 'bands':['B8', 'B11', 'B12']}
      vi_bands = ['B8', 'B11', 'B12']
    elif vi_index == "EVI":
      def vi_func(image):
        evi = image.expression(
          '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename(vi_index)
        return evi.set('system:time_start', image.get('system:time_start'))
      vi_palatte = {'min':0, 'max':1, 'palette': ['ffffff', '0b6b07']}
      vi_bands = vi_index  
    elif vi_index == "LSWI":
      def vi_func(image):
        return image.normalizedDifference(['B8', 'B11']).rename(vi_index)
      vi_palatte = {'min':-1, 'max':1, 'palette': ['ffffff', '16499e']}  
      vi_bands = vi_index

    s2_dataset = obtainS2Dataset(study_area, date_start, date_end)
    s2_dataset = s2_dataset.map(vi_func)
    if checkbox_mask.value == False:
      fir_img = s2_dataset.median().clip(study_area)
    else:
      fir_img = s2_dataset.median().clip(study_area).updateMask(maize_img)

    # output_layer = geemap.ee_tile_layer(fir_img, vi_palatte, vi_index)
    return fir_img, vi_palatte, vi_index
    # image_res = 0    
    # map.addLayer(fir_img.select(vi_bands), vi_palatte, vi_index)

# ****** this function is used to display the second image on a specific map ******
def obtain_SecImage():
  vi_index = second_image.value
  if vi_index != None:
    if vi_index == "Maize classification":
      image_res = maize_img
      # print(image_res)
      vi_palatte = {'min':0, 'max':1, 'palette': ['ffffff', 'ffab40']}
    elif vi_index == "Yield result":
      # print(vi_index)
      image_res = maize_img
      vi_palatte = {'min':0, 'max':1, 'palette': ['ffffff', 'ffab40']}

    # map.addLayer(image_res, vi_palatte, vi_index)
    # output_layer = geemap.ee_tile_layer(image_res, vi_palatte, vi_index)
    return image_res, vi_palatte, vi_index
# --------------------------------------------------------------------------------------------------------------------------------------

# ****** the event of submit button ******
# --------------------------------------------------------------------------------------------------------------------------------------
def submit_clicked(button):
  Map.layers = Map.layers[0:3]
  # output_widget.clear_output()
  vi_index_0 = first_image.value
  vi_index_1 = second_image.value
  # print(vi_index_0, vi_index_1)

  if checkbox_split.value == False:
    if vi_index_0 != "None":
      fir_img, vi_palatte_0, vi_index_0 = obtain_FirImage()
      Map.addLayer(fir_img, vi_palatte_0, vi_index_0)
    if vi_index_1 != "None":  
      sec_img, vi_palatte_1, vi_index_1 = obtain_SecImage()
      Map.addLayer(sec_img, vi_palatte_1, vi_index_1)
    
    # display_FirImage(Map)
    # display_SecImage(Map)
  elif checkbox_split.value == True:
    if vi_index_0 != "None":
      fir_img, vi_palatte_0, vi_index_0 = obtain_FirImage()
      fir_layer = geemap.ee_tile_layer(fir_img, vi_palatte_0, vi_index_0)
    else:
      fir_layer = 'HYBRID'
    
    if vi_index_1 != "None":
      sec_img, vi_palatte_1, vi_index_1 = obtain_SecImage()
      sec_layer = geemap.ee_tile_layer(sec_img, vi_palatte_1, vi_index_1)
    else:
      sec_layer = 'HYBRID'  

    Map.split_map(fir_layer, sec_layer)  
    # display_FirImage(Map)  

    # Map.addLayer(s2_img, tc_vis, "s2_img")

# --------------------------------------------------------------------------------------------------------------------------------------


In [10]:
# ###### define the study area and corresponding maize mask ######
# --------------------------------------------------------------------------------------------------------------------------------------
# imgCol_Countries = ee.FeatureCollection("USDOS/LSIB/2017")
# study_area = imgCol_Countries.filter(ee.Filter.eq('COUNTRY_NA', 'Ghana'))
cities = ee.FeatureCollection("FAO/GAUL/2015/level2")
ghana_district = cities.filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))
northern_district = ghana_district.filter(ee.Filter.eq('ADM1_NAME', 'Northern'))
study_area = northern_district

maize_imgCol = ee.ImageCollection('users/xianda19/classification_result/2021/Ghana/maize_20210501_20211011_100percentSamples')
maize_img = maize_imgCol.mosaic().selfMask()
outline = ee.Image().paint(study_area, 0, 3)
# --------------------------------------------------------------------------------------------------------------------------------------

Map = geemap.Map()
Map.add_control(output_control)
Map.addLayer(outline, {'palette': '000000'}, 'study area')
# Map.addLayer(maize_img, visParam, "maize_img")
Map.centerObject(study_area, 8)

full_widgets = widgets.VBox([
  widgets.HBox([Map]),
  widgets.HBox([startDate, first_image, checkbox_mask], layout=Layout(height="40px")),
  widgets.HBox([endDate, second_image, checkbox_split], layout=Layout(height="40px")),
  widgets.HBox([submit], layout=Layout(height="40px"))
])
submit.on_click(submit_clicked)

display(full_widgets)
# display(full_widgets)

# **User guide of the interactive system**
## **Step 1:**
Determine the temporal range (start date and end date) of the first image through the time controller.

## **Step 2:**
Choose the type of first image and second image through dropdown widgets (First image and Second image). \
**First image type**: \
vegetation index (including ndvi, evi, and lswi),composite images (including red/green/blue, nir/red/green, and nir/swir1/swir2).\
**Second image type:**
maize classification and yield result

## **Step 3:**
When you tick the checkbox named "Apply maize mask", the first image will be masked by the maize map. And when you tick the "Split window" checkbox, the first and second images will be visualized on the interface at the same time.
